In [7]:

import pandas as pd
import numpy as np
import os
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import scipy.stats
from pathlib import Path


def PCC_score(y_pred,y_true):
    diff_pred,diff_true=y_pred-np.mean(y_pred),y_true-np.mean(y_true)
    return np.sum(diff_pred*diff_true)/np.sqrt(np.sum(diff_pred**2)*np.sum(diff_true**2))

# MAE = mean_absolute_error(y_true, tool_data_list[i])
# MSE = mean_squared_error(y_true, tool_data_list[i])
# RMSE = np.sqrt(MSE)
# r2 = r2_score(y_true, tool_data_list[i])
# PCC = PCC_score(y_true, tool_data_list[i])
# tau = scipy.stats.kendalltau(y_true, tool_data_list[i])[0]

allele_size_cutoff = 1
prediction_dir = '/mnt/zt/Dopaap/calculate_webtools_score/compare_HPV16_dataset/processed_for_calculate'
results_score_dir = '/mnt/zt/Dopaap/calculate_webtools_score/compare_HPV16_dataset/results_score'

In [8]:
def calculate_allele_score():    
    df_results_score = pd.DataFrame(columns=['allele','allele_size', 'tool', 'r2', 'PCC', 'tau', 'MAE', 'MSE', 'RMSE'])

    for file in os.listdir(prediction_dir):
        allele = Path(file).stem
        df_allele_prediction = pd.read_csv(os.path.join(prediction_dir, file))

        if not df_allele_prediction.columns.str.contains('Dopaap').any():
            continue
        
        if len(df_allele_prediction) < allele_size_cutoff:
            continue
        
        y_true = df_allele_prediction['Normalized_QM']
        
        y_pred_list = [tool_score for tool_score in df_allele_prediction.columns if '_Normalized' in tool_score]
        for tool_score in y_pred_list:
            y_pred = df_allele_prediction[tool_score]
            MAE = mean_absolute_error(y_true, y_pred)
            MSE = mean_squared_error(y_true, y_pred)
            RMSE = np.sqrt(MSE)
            r2 = r2_score(y_true, y_pred)
            PCC = PCC_score(y_true, y_pred)
            tau = scipy.stats.kendalltau(y_true, y_pred)[0]
            
            tool_name = tool_score.split('_')[0]
            df_results_score = df_results_score.append({'allele': allele, 'allele_size': len(df_allele_prediction), 'tool': tool_name, 'r2': r2, 'PCC': PCC, 'tau': tau , 'MAE': MAE, 'MSE': MSE, 'RMSE': RMSE}, ignore_index=True)
    df_results_score.to_csv(os.path.join(results_score_dir, 'results_score.csv'), index=False)        


In [9]:
def calculate_total_score(): 
    df_results_score = pd.read_csv(os.path.join(results_score_dir, 'results_score.csv'))
    df_total_prediction = pd.DataFrame()

    for file in os.listdir(prediction_dir):
        allele = Path(file).stem
        df_allele_prediction = pd.read_csv(os.path.join(prediction_dir, file))
        
        if not df_allele_prediction.columns.str.contains('Dopaap').any():
            continue
        
        if len(df_allele_prediction) < allele_size_cutoff:
            continue
        
        df_total_prediction = pd.concat([df_total_prediction, df_allele_prediction])
        
    df_total_prediction.to_csv(os.path.join(results_score_dir, 'total_prediction.csv'), index=False)

    y_pred_list = [tool_score for tool_score in df_total_prediction.columns if '_Normalized' in tool_score]

    for tool_score in y_pred_list:
        df_tool_prediction = df_total_prediction[~pd.isna(df_total_prediction[tool_score])]
        
        y_true = df_tool_prediction['Normalized_QM']
        y_pred = df_tool_prediction[tool_score]
        
        MAE = mean_absolute_error(y_true, y_pred)
        MSE = mean_squared_error(y_true, y_pred)
        RMSE = np.sqrt(MSE)
        r2 = r2_score(y_true, y_pred)
        PCC = PCC_score(y_true, y_pred)
        tau = scipy.stats.kendalltau(y_true, y_pred)[0]
        
        tool_name = tool_score.split('_')[0]
        
        df_results_score = df_results_score.append({'allele': 'Total allele', 'allele_size': len(df_tool_prediction), 'tool': tool_name, 'r2': r2, 'PCC': PCC, 'tau': tau , 'MAE': MAE, 'MSE': MSE, 'RMSE': RMSE}, ignore_index=True)
    df_results_score.to_csv(os.path.join(results_score_dir, 'results_score.csv'), index=False)        



In [10]:
calculate_allele_score()
calculate_total_score()